# Exploring SNSF Public Data

## Foreword

The [Swiss National Science Foundation (SNSF)](http://www.snf.ch/en/Pages/default.aspx) is the leading public science funding agency in Switzerland.  In _GrantExport_, the SNSF provides detailed information about their approved grants.  In _PersonExport_, it also provides some information about the people associated to the grants.  Other tables provide further data about scientific publications, outputs, and collaborations reported by the grant applicants.  Please find the datasets and documentation [here](http://p3.snf.ch/Pages/DataAndDocumentation.aspx).

Prior to exploring the data, I performed a number of cleaning and tidying operations on the _GrantExport_ and _PersonExport_ datasets (see [cleaning.html](./cleaning.html)).

In [ ]:
# boilerplate
import numpy as np
import pandas as pd
import matplotlib as plt
import seaborn as sns
from plotnine import *
sns.set_style('whitegrid')

# open tidy datasets
grants=pd.read_feather('data/grant.feather')
grantees=pd.read_feather('data/grantee.feather')
people=pd.read_feather('data/person.feather')